In [102]:
import pyarrow as py
import pandas as pd
import numpy as np
import glob

# Path to the folder where your Parquet files are stored
folder_path = '/content/*.parquet'

# Get a list of all Parquet file paths in the folder
parquet_files = glob.glob(folder_path)

# Read all Parquet files into a list of DataFrames
dfs = [pd.read_parquet(file) for file in parquet_files]

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)


effective_cur_columns =['line_item_resource_id','bill_payer_account_id','bill_billing_period_start_date','bill_billing_period_end_date','line_item_usage_account_id', 'line_item_line_item_type', 'line_item_usage_start_date', 'line_item_usage_end_date',
               'line_item_product_code','product_product_name', 'product_instance_family', 'product_instance_type', 'product_instance_type_family','line_item_usage_type', 'line_item_operation', 'line_item_availability_zone',
               'line_item_usage_amount','line_item_normalization_factor', 'line_item_normalized_usage_amount','line_item_unblended_rate', 'line_item_unblended_cost', 'pricing_term', 'pricing_unit','pricing_offering_class', 'pricing_purchase_option',
               'reservation_amortized_upfront_cost_for_usage', 'reservation_amortized_upfront_fee_for_billing_period', 'reservation_effective_cost', 'reservation_end_time','reservation_normalized_units_per_reservation',
               'reservation_number_of_reservations', 'reservation_recurring_fee_for_usage', 'reservation_reservation_a_r_n', 'reservation_start_time', 'reservation_subscription_id', 'reservation_total_reserved_normalized_units',
               'reservation_total_reserved_units', 'reservation_units_per_reservation', 'reservation_unused_amortized_upfront_fee_for_billing_period', 'reservation_unused_normalized_unit_quantity', 'reservation_unused_quantity',
               'reservation_unused_recurring_fee', 'reservation_upfront_value','product_operating_system','product_region']
# projection on the df to get data of only selected colmns
df = df.loc[:,effective_cur_columns]

# filtering df for only selected product names
product_names = [
    'Amazon Elastic Compute Cloud',
    'Amazon Relational Database Service',
    'Amazon ElastiCache',
    'Amazon OpenSearch Service',
    'Amazon Elasticsearch Service',
    'Amazon Redshift',
    'Amazon DynamoDB'
]

df = df[df['product_product_name'].isin(product_names)]

df



,line_item_resource_id,bill_payer_account_id,bill_billing_period_start_date,bill_billing_period_end_date,line_item_usage_account_id,line_item_line_item_type,line_item_usage_start_date,line_item_usage_end_date,line_item_product_code,product_product_name,...,reservation_total_reserved_normalized_units,reservation_total_reserved_units,reservation_units_per_reservation,reservation_unused_amortized_upfront_fee_for_billing_period,reservation_unused_normalized_unit_quantity,reservation_unused_quantity,reservation_unused_recurring_fee,reservation_upfront_value,product_operating_system,product_region
0,,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,2976.00000000000,744.0000000000,744.0000000000,22.336986,2976.0,744.0,22.32,263.0,,us-east-1
1,,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,5952.00000000000,1488.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,446.0,,us-east-1
2,,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,2976.00000000000,744.0000000000,744.0000000000,0.150457,8.0,2.0,0.15,659.0,,us-east-1
3,,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,372.00000000000,744.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,0.0,,us-east-1
4,,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,1116.00000000000,2232.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,90.0,,us-east-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334879,i-0342529a9832468d5,836060457634,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,836060457634,Usage,2018-11-30 23:00:00+00:00,2018-12-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,,,,0.000000,0.0,0.0,0.00,0.0,Linux,eu-west-2
334880,i-0342529a9832468d5,836060457634,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,836060457634,Usage,2018-11-30 23:00:00+00:00,2018-12-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,,,,0.000000,0.0,0.0,0.00,0.0,,eu-west-2
334881,i-0342529a9832468d5,836060457634,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,836060457634,Usage,2018-11-30 23:00:00+00:00,2018-12-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,,,,0.000000,0.0,0.0,0.00,0.0,,eu-west-2
334882,vol-0092ffd396f37de25,836060457634,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,836060457634,Usage,2018-11-30 23:00:00+00:00,2018-12-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,...,,,,0.000000,0.0,0.0,0.00,0.0,,eu-west-2


In [103]:
# getting DiscountedUsage line_item_line_item_type to fetch pricing_offering_class, pricing_purchase_option, reservation_reservation_a_r_n
# Filter the DataFrame where 'line_item_line_item_type' is 'DiscountedUsage'
discounted_usage_df = df[df['line_item_line_item_type'] == 'DiscountedUsage']

# Group by the required columns and get the unique combinations
discounted_usage_df = discounted_usage_df[['pricing_offering_class', 'pricing_purchase_option', 'reservation_reservation_a_r_n']].drop_duplicates()

discounted_usage_df


,pricing_offering_class,pricing_purchase_option,reservation_reservation_a_r_n
13,standard,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28164,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28166,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28167,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28168,standard,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28169,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28170,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28171,convertible,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...


In [104]:
# Filter the DataFrame where 'line_item_line_item_type' is 'DiscountedUsage'

on_demand_that_can_be_reserved = 'NodeUsage|ReadRequestUnits|WriteRequestUnits|InstanceUsage|BoxUsage'

usage_df = df[df['line_item_line_item_type'] == 'Usage']


# Filter the DataFrame
usage_df = usage_df[usage_df['line_item_usage_type'].str.contains(on_demand_that_can_be_reserved, na=False)]

usage_df['line_item_usage_amount'] = usage_df['line_item_usage_amount'].astype(float)
usage_df['line_item_normalized_usage_amount'] = usage_df['line_item_normalized_usage_amount'].astype(float)
usage_df['line_item_normalization_factor'] = usage_df['line_item_normalization_factor'].astype(float)
usage_df.loc[usage_df['line_item_normalization_factor'] == 0, 'line_item_normalization_factor'] = 1
usage_df['line_item_normalization_factor'].fillna(1).astype(int)
usage_df.loc[usage_df['line_item_normalization_factor'] == '', 'line_item_normalization_factor'] = 1
usage_df['line_item_unblended_cost'] = usage_df['line_item_unblended_cost'].astype(float)

usage_df = usage_df.groupby(
    [   'line_item_resource_id',
        'bill_payer_account_id',
        'product_product_name',
        'line_item_product_code',
        'product_region',
        'line_item_usage_type',
        'product_instance_type_family',
        'product_instance_type',
        'line_item_operation',
        'line_item_line_item_type',
        'line_item_normalization_factor'
    ]
).agg(
    line_item_usage_amount=('line_item_usage_amount', 'sum'),
    line_item_unblended_cost=('line_item_unblended_cost', 'count')
).reset_index()
usage_df['line_item_normalized_usage_amount'] = usage_df['line_item_usage_amount'].astype(float) * usage_df['line_item_normalization_factor'].astype(float)
usage_df
usage_df.to_csv('ondemand_usage.csv', index=False)

In [108]:
#  reservation DF

ri_fee = 'RIFee'

line_item_line_item_type = 'line_item_line_item_type'
reservation_reservation_a_r_n = 'reservation_reservation_a_r_n'
bill_billing_period_month = 'bill_billing_period_month'

effective_reservation_columns = ['bill_payer_account_id','line_item_usage_account_id','product_product_name','line_item_product_code',
'line_item_usage_type','line_item_line_item_type','reservation_reservation_a_r_n','line_item_unblended_rate',
'product_instance_type_family','product_operating_system','line_item_operation','reservation_start_time','reservation_end_time','bill_billing_period_start_date','bill_billing_period_end_date',
'reservation_total_reserved_normalized_units','reservation_unused_normalized_unit_quantity','line_item_normalization_factor','product_region']

reservation_df = df.loc[:,effective_reservation_columns]


# getting reservation purchased
reservation_df = reservation_df[reservation_df[line_item_line_item_type] == ri_fee]

reservation_df['reservation_total_reserved_normalized_units'] = reservation_df['reservation_total_reserved_normalized_units'].astype(float)

reservation_df['reservation_unused_normalized_unit_quantity'] = reservation_df['reservation_unused_normalized_unit_quantity'].astype(float)

reservation_df['reservation_start_time'] = pd.to_datetime(reservation_df['reservation_start_time'])
reservation_df['reservation_end_time'] = pd.to_datetime(reservation_df['reservation_end_time'])

reservation_df['bill_billing_period_start_date'] = pd.to_datetime(reservation_df['bill_billing_period_start_date'])
reservation_df['bill_billing_period_end_date'] = pd.to_datetime(reservation_df['bill_billing_period_end_date'])

# Calculate the total hours between the start and end date
reservation_df['bill_billing_period_total_hours'] = (reservation_df['bill_billing_period_end_date'] - reservation_df['bill_billing_period_start_date']).dt.total_seconds() / 3600

# Calculate the total YEAR between the start and end date
reservation_df['reservation_purchase_term'] = ((reservation_df['reservation_end_time'] - reservation_df['reservation_start_time']).dt.days / 365).round()

# Calculate billing period month
reservation_df['bill_billing_period_month'] = (reservation_df['bill_billing_period_start_date'].dt.month)

reservation_df['product_instance_type_family'] = reservation_df['line_item_usage_type'].str.split(':').str[-1].str.split('.').str[0]
reservation_df['product_instance_type'] = reservation_df['line_item_usage_type'].str.split(':').str[-1]


reservation_df['line_item_normalization_factor'] = reservation_df['line_item_normalization_factor'].astype(float)
reservation_df.loc[reservation_df['line_item_normalization_factor'] == 0, 'line_item_normalization_factor'] = 1

# get columns 'pricing_offering_class', 'pricing_purchase_option' from discounted_usage_df
reservation_df = reservation_df.merge(discounted_usage_df, on='reservation_reservation_a_r_n', how='left')

reservation_df['pricing_offering_class'] = reservation_df['pricing_offering_class'].fillna('Unknown')

reservation_df['pricing_purchase_option'] = reservation_df['pricing_purchase_option'].fillna('Unknown')


reservation_df = reservation_df.groupby(
    [
        'bill_payer_account_id',
        'reservation_reservation_a_r_n',
        'product_product_name',
        'line_item_product_code',
        'line_item_usage_type',
        'product_region',
        'product_instance_type_family',
        'product_instance_type',
        'pricing_offering_class',
        'pricing_purchase_option',
        'line_item_operation',
        'line_item_line_item_type',
        'reservation_purchase_term',
        'line_item_unblended_rate',
        'line_item_normalization_factor'
    ]
).agg(
    reservation_total_reserved_normalized_units=('reservation_total_reserved_normalized_units', 'sum'),
    reservation_unused_normalized_unit_quantity=('reservation_unused_normalized_unit_quantity', 'sum'),
    number_of_months=('bill_billing_period_month', 'count')
).reset_index()



# calculate reservation_utilization_cost,reservation_wastage_cost
reservation_df['reservation_used_normalized_unit_quantity'] = reservation_df['reservation_total_reserved_normalized_units'].astype(float) - reservation_df['reservation_unused_normalized_unit_quantity'].astype(float)
reservation_df['reservation_wastage_cost'] = (reservation_df['reservation_unused_normalized_unit_quantity']).astype(float) * (reservation_df['line_item_unblended_rate']).astype(float)
reservation_df['reservation_utilization_cost'] = (reservation_df['reservation_total_reserved_normalized_units']).astype(float) * (reservation_df['line_item_unblended_rate']).astype(float)
reservation_df['reservation_wastage_cost'] = (reservation_df['reservation_unused_normalized_unit_quantity']).astype(float) * (reservation_df['line_item_unblended_rate']).astype(float)
reservation_df['reservation_utilization_percent'] = (reservation_df['reservation_used_normalized_unit_quantity']).astype(float) / (reservation_df['reservation_total_reserved_normalized_units']).astype(float) * 100
reservation_df['reservation_wastage_percent'] = (reservation_df['reservation_unused_normalized_unit_quantity']).astype(float) / (reservation_df['reservation_total_reserved_normalized_units']).astype(float) * 100


reservation_df.rename(columns={'line_item_normalization_factor': 'reservation_normalization_factor'}, inplace=True)

# dorting data based on ['reservation_reservation_a_r_n','bill_billing_period_month']
reservation_df = reservation_df.sort_values(by=[reservation_reservation_a_r_n])

# Rename the column in-place




# group usage_df by those columns and count occurrences
usage_df_instance_count = usage_df.groupby([
    'product_product_name',
    'line_item_product_code',
    'product_region',
    'product_instance_type',
    'line_item_normalization_factor'
]).size().reset_index(name='not_utilize_usage_instance_count')


usage_df_instance_count.rename(columns={'line_item_normalization_factor': 'usage_normalization_factor'}, inplace=True)



# Merge reservation_df with the usage_df_instance_count
reservation_df = reservation_df.merge(usage_df_instance_count, on=[
    'product_product_name',
    'line_item_product_code',
    'product_region',
    'product_instance_type'
], how='left')

# Step 4: Fill NaNs with 0 (if there was no match in df2)
reservation_df['not_utilize_usage_instance_count'] = reservation_df['not_utilize_usage_instance_count'].fillna(0).astype(int)
reservation_df['usage_normalization_factor'] = reservation_df['usage_normalization_factor'].fillna(1).astype(int)



# Define conditions

cond1 = (
    (reservation_df['reservation_unused_normalized_unit_quantity'] > 0) &
    (reservation_df['not_utilize_usage_instance_count'] > 0) &
    (reservation_df['usage_normalization_factor'] < reservation_df['reservation_normalization_factor'])
)


cond2 = (
    (reservation_df['not_utilize_usage_instance_count'] > 0) &
    (reservation_df['usage_normalization_factor'] < reservation_df['reservation_normalization_factor'])
)

cond3 = reservation_df['reservation_unused_normalized_unit_quantity'] > 0

# Define corresponding values
choices = [
    "ri_inefficient",     # this should be first, as it's a more specific case
    "ri_underutilized",
    "ri_wastage"
]

# Order of conditions matters — most specific to most general
conditions = [cond1, cond2, cond3]

# Create the column
reservation_df['manual_recommendation'] = np.select(conditions, choices, default="ri_optimized")



reservation_df.to_csv('reservation_utilization_wastage.csv', index=False)

reservation_df





,bill_payer_account_id,reservation_reservation_a_r_n,product_product_name,line_item_product_code,line_item_usage_type,product_region,product_instance_type_family,product_instance_type,pricing_offering_class,pricing_purchase_option,...,reservation_unused_normalized_unit_quantity,number_of_months,reservation_used_normalized_unit_quantity,reservation_wastage_cost,reservation_utilization_cost,reservation_utilization_percent,reservation_wastage_percent,usage_normalization_factor,not_utilize_usage_instance_count,manual_recommendation
0,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.micro,us-east-1,t2,t2.micro,convertible,No Upfront,...,22.999722,3,1081.000278,0.190898,9.16320,97.916692,2.083308,0,189,ri_inefficient
1,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.small,us-east-1,t2,t2.small,standard,Partial Upfront,...,0.000000,3,2208.000000,0.000000,15.01440,100.000000,0.000000,1,0,ri_optimized
2,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:c5.large,us-east-1,c5,c5.large,standard,Partial Upfront,...,0.000000,3,17664.000000,0.000000,459.26400,100.000000,0.000000,4,5,ri_optimized
3,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.micro,us-east-1,t2,t2.micro,standard,No Upfront,...,503.160139,3,494.000000,5.937290,11.76649,49.540689,50.459311,0,189,ri_inefficient
4,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:c4.large,us-east-1,c4,c4.large,Unknown,Unknown,...,8832.000000,3,0.000000,264.960000,264.96000,0.000000,100.000000,1,0,ri_wastage
5,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.nano,us-east-1,t2,t2.nano,standard,Partial Upfront,...,0.000000,3,1104.000000,0.000000,1.87680,100.000000,0.000000,1,0,ri_optimized
6,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.micro,us-east-1,t2,t2.micro,standard,Partial Upfront,...,0.000000,3,3312.000000,0.000000,11.26080,100.000000,0.000000,0,189,ri_underutilized
7,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:t2.micro,us-east-1,t2,t2.micro,standard,No Upfront,...,0.000000,3,1104.000000,0.000000,7.94880,100.000000,0.000000,0,189,ri_underutilized
8,836060457634,arn:aws:ec2:us-east-1:836060457634:reserved-in...,Amazon Elastic Compute Cloud,AmazonEC2,HeavyUsage:m5.large,us-east-1,m5,m5.large,standard,Partial Upfront,...,8.000000,3,8824.000000,0.600000,662.40000,99.909420,0.090580,4,1,ri_wastage
